In [57]:
import gurobipy as gp
from gurobipy import GRB
import json
from pathlib import Path
import pandas as pd

In [58]:
dimension = "5_30_instances"
folder_projects = Path("instances_projects")
filename_projects = "generic_5_30_projects_4.csv"
filepath_projects = folder_projects / dimension / filename_projects
folder_students = Path("instances_students")
filename_students = "generic_5_30_students_4.csv"
filepath_students = folder_students / dimension / filename_students
projects_info = pd.read_csv(filepath_projects)
students_info = pd.read_csv(filepath_students)
students_info["fav_partners"] = students_info["fav_partners"].apply(lambda x: set(json.loads(x)))
students_info["project_prefs"] = students_info["project_prefs"].apply(lambda x: tuple(json.loads(x)))

In [59]:
print(projects_info)

                           name  desired#groups  max#groups  ideal_group_size  \
0                     Marketing               3           5                 3   
1         Innovation Management               4           6                 2   
2        Payroll Administration               3           6                 3   
3             Consumer Behavior               4           5                 4   
4  Business Information Systems               3           4                 4   

   min_group_size  max_group_size  pen_groups  pen_size  
0               2               4           3         3  
1               1               4           2         1  
2               2               4           3         2  
3               1               5           2         2  
4               1               5           3         3  


In [60]:
print(students_info)

                name  fav_partners    project_prefs
0        Jose Wright   {1, 18, 14}  (1, 2, 3, 2, 3)
1     Raymond Thomas    {0, 9, 25}  (1, 2, 3, 2, 3)
2        Betty James    {9, 21, 1}  (1, 1, 2, 1, 2)
3       Albert Moore  {16, 27, 20}  (1, 3, 2, 1, 2)
4     Kathryn Nelson    {1, 3, 21}  (2, 2, 3, 2, 2)
5      Paul Martinez   {19, 21, 7}  (0, 2, 2, 0, 2)
6      Sandra Hughes   {1, 28, 17}  (1, 2, 2, 2, 2)
7      Keith Mendoza   {18, 29, 5}  (0, 1, 2, 1, 2)
8      Raymond Lewis   {11, 29, 7}  (0, 1, 2, 1, 2)
9     Kimberly Brown   {1, 20, 23}  (1, 1, 3, 1, 2)
10  Theresa Anderson     {9, 3, 6}  (1, 2, 2, 1, 2)
11     Elijah Taylor   {8, 26, 15}  (1, 2, 2, 1, 2)
12          Jack Cox   {10, 5, 23}  (1, 2, 1, 1, 2)
13      John Johnson   {5, 29, 23}  (1, 1, 1, 1, 2)
14  Dorothy Anderson    {0, 18, 3}  (2, 2, 3, 2, 3)
15       Amanda Ross  {25, 27, 29}  (1, 3, 2, 3, 2)
16        Keith Long    {19, 2, 3}  (1, 2, 2, 2, 2)
17         Lori Hill   {8, 18, 12}  (1, 1, 2, 1, 2)
18      Kare

In [61]:
reward_bilateral = 2
penalty_unassignment = 3

In [62]:

model = gp.Model()

Set parameter Username
Set parameter LicenseID to value 2654427
Academic license - for non-commercial use only - expires 2026-04-19


In [63]:
project_ids = range(len(projects_info))
student_ids = range(len(students_info))

projects_group_ids = {
    project_id: range(projects_info["max#groups"][project_id])
    for project_id in project_ids
}


In [64]:
x = model.addVars(
    (
        (project_id, group_id, student_id)
        for project_id in project_ids
        for group_id in projects_group_ids[project_id]
        for student_id in student_ids
    ),
    vtype=GRB.BINARY,
    name="assign",
)
model.update()

In [65]:
y = model.addVars(
    (
        (project_id, group_id)
        for project_id in project_ids
        for group_id in projects_group_ids[project_id]
    ),
    vtype=GRB.BINARY,
    name="establish_group",
)

In [66]:
favorite_partners_students = students_info["fav_partners"].tolist()
mutual_pairs = {
    (student_id, partner_id)
    for student_id, favorite_partners in enumerate(favorite_partners_students)
    for partner_id in favorite_partners
    if student_id < partner_id and
    student_id in favorite_partners_students[partner_id]
}

In [67]:
z = model.addVars(mutual_pairs, vtype=GRB.BINARY, name="not_realize_bilateral_cooperation_wish")

In [68]:
v = model.addVars(student_ids, name = "student_unassigned")

In [69]:
gs_surplus = model.addVars(
    (
        (project_id, group_id)
        for project_id in project_ids
        for group_id in projects_group_ids[project_id]
    ),
    name="group_size_surplus",
)

In [70]:
gs_deficit = model.addVars(
    (
        (project_id, group_id)
        for project_id in project_ids
        for group_id in projects_group_ids[project_id]
    ),
    name="group_size_deficit",
)

In [71]:
project_preferences = {
    (student_id, project_id): students_info["project_prefs"][student_id][project_id]
    for student_id in student_ids
    for project_id in project_ids
}

In [72]:
rewards_student_project_preference = gp.quicksum(
    project_preferences[student_id, project_id] * x[project_id, group_id, student_id]
    for project_id in project_ids
    for group_id in projects_group_ids[project_id]
    for student_id in student_ids
)

rewards_bilateral_cooperation_wish_realized = reward_bilateral * gp.quicksum(
    1 - z[*mutual_pair] for mutual_pair in mutual_pairs
)

penalties_student_unassigned = penalty_unassignment * gp.quicksum(v.values())

penalties_more_groups_than_offered = gp.quicksum(
    projects_info["pen_groups"][project_id] * y[project_id, group_id]
    for project_id in project_ids
    for group_id in projects_group_ids[project_id]
    if group_id >= projects_info["desired#groups"][project_id]
)

penalties_not_ideal_group_size = gp.quicksum(
    projects_info["pen_size"][project_id]
    * (gs_surplus[project_id, group_id] + gs_deficit[project_id, group_id])
    for project_id in project_ids
    for group_id in projects_group_ids[project_id]
)

model.setObjective(
    rewards_student_project_preference
    + rewards_bilateral_cooperation_wish_realized
    - penalties_student_unassigned
    - penalties_more_groups_than_offered
    - penalties_not_ideal_group_size,
    sense=GRB.MAXIMIZE,
)

In [73]:
model.addConstrs(
    (x.sum("*", "*", student_id) + v[student_id] == 1 for student_id in student_ids),
    name="penalty_if_unassigned",
)
model.update()

In [74]:
model.addConstrs(
    (
        y[project_id, group_id] <= y[project_id, group_id - 1]
        for project_id in project_ids
        for group_id in projects_group_ids[project_id]
        if group_id > 0
    ), name="only_consecutive_group_ids"
)
model.update()

In [75]:
model.addConstrs(
    (
        x.sum(project_id, group_id, "*")
        >= projects_info["min_group_size"][project_id] * y[project_id, group_id]
        for project_id in project_ids
        for group_id in projects_group_ids[project_id]
    ), name="ensure_min_group_size"
)
model.update()

In [76]:
model.addConstrs(
    (
        x.sum(project_id, group_id, "*")
        <= projects_info["max_group_size"][project_id] * y[project_id, group_id]
        for project_id in project_ids
        for group_id in projects_group_ids[project_id]
    ), name="cap_group_size_at_max"
)
model.update()

In [77]:
model.addConstrs(
    (
        gs_surplus[project_id, group_id]
        >= x.sum(project_id, group_id, "*")
        - projects_info["ideal_group_size"][project_id]
        for project_id in project_ids
        for group_id in projects_group_ids[project_id]
    ), name="ensure_correct_group_size_surplus"
)
model.update()

In [78]:
model.addConstrs(
    (
        gs_deficit[project_id, group_id]
        >= projects_info["ideal_group_size"][project_id]
        - x.sum(project_id, group_id, "*")
        - projects_info["max_group_size"][project_id] * (1 - y[project_id, group_id])
        for project_id in project_ids
        for group_id in projects_group_ids[project_id]
    ), name="ensure_correct_group_size_deficit"
)
model.update()

In [79]:
max_num_groups = max(projects_info["max#groups"])


unique_group_identifiers = {
    (project_id, group_id): project_id + group_id / max_num_groups
    for project_id in project_ids
    for group_id in projects_group_ids[project_id]
}

In [80]:
num_projects = len(projects_info)

model.addConstrs(
    (
        z[first_student_id, second_student_id] * num_projects
        >= sum(
            unique_group_identifiers[project_id, group_id]
            * (
                x[project_id, group_id, first_student_id]
                - x[project_id, group_id, second_student_id]
            )
            for project_id in project_ids
            for group_id in projects_group_ids[project_id]
        )
        for (first_student_id, second_student_id) in mutual_pairs
    ), name="ensure_correct_inidicator_different_group_1"
)
model.update()

In [81]:
model.addConstrs(
    (
        z[first_student_id, second_student_id] * num_projects
        >= sum(
            unique_group_identifiers[project_id, group_id]
            * (
                x[project_id, group_id, second_student_id]
                - x[project_id, group_id, first_student_id]
            )
            for project_id in project_ids
            for group_id in projects_group_ids[project_id]
        )
        for (first_student_id, second_student_id) in mutual_pairs
    ), name="ensure_correct_inidicator_different_group_2"
)
model.update()

In [82]:
#model.Params.TimeLimit = 60
model.optimize()

Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i5-12500H, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 205 rows, 913 columns and 6652 nonzeros
Model fingerprint: 0x3671536b
Variable types: 82 continuous, 831 integer (831 binary)
Coefficient statistics:
  Matrix range     [2e-01, 5e+00]
  Objective range  [1e+00, 3e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective -40.0000000
Presolve removed 26 rows and 0 columns
Presolve time: 0.03s
Presolved: 179 rows, 913 columns, 5872 nonzeros
Variable types: 0 continuous, 913 integer (861 binary)

Root relaxation: objective 1.174951e+02, 284 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Ti

In [ ]:
num_assigned = 0
for project_id in project_ids:
    print("\n")
    print(projects_info["name"][project_id])
    for group_id in projects_group_ids[project_id]:
        print("\nGroup", group_id + 1)
        for student_id in student_ids:
            if x[project_id, group_id, student_id].X > 0.5:
                print(students_info["name"][student_id], student_id)
                num_assigned += 1

print("Number of assigned students:", num_assigned)



Marketing

Group 1

Group 2

Group 3

Group 4

Group 5


Innovation Management

Group 1
Albert Moore 3
Amanda Ross 15
Wayne Gray 27

Group 2
Keith Long 16
Kevin Lee 19

Group 3
Jack Cox 12
Eric Morales 23

Group 4
Sandra Hughes 6
Caleb Rogers 28

Group 5

Group 6


Payroll Administration

Group 1
Theresa Anderson 10
Emily Taylor 24
Sophia Lopez 25

Group 2
Betty James 2
Kathryn Nelson 4
Barbara Brown 21

Group 3
Raymond Thomas 1
Kimberly Brown 9
Doris Ramos 20

Group 4

Group 5

Group 6


Consumer Behavior

Group 1

Group 2

Group 3

Group 4

Group 5


Business Information Systems

Group 1
Paul Martinez 5
Keith Mendoza 7
John Johnson 13
Logan Lopez 29

Group 2
Raymond Lewis 8
Elijah Taylor 11
Linda Carter 22
Maria Wilson 26

Group 3
Jose Wright 0
Dorothy Anderson 14
Lori Hill 17
Karen Rogers 18

Group 4
Number of assigned: students: 30


In [84]:
model.dispose()
gp.disposeDefaultEnv()

Freeing default Gurobi environment
